In [1]:
import numpy as np
from matplotlib import pyplot as plt

import metpy
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



First make sure we are merging separete sweeps into one volume. We do this with RadxConvert from lrose. Installation instructions are [here](https://github.com/NCAR/lrose-core/blob/master/docs/build/LROSE_cmake_build.auto.md). We will also need netcdf (ncdump specifically) to run the given script. Run the following in terminal, it requires input.